In [9]:
dimension = 3
perimeter = 10
sensitivity = 1.5
jsonfilename = "myfile2.json"

In [54]:
def main() -> None:
    output_data = []
    for iteration in range(10000):
        B = randomMatrix(dimension, perimeter)
        data = abnormality_check(B)
        if data != None:
            output_data.append(abnormality_check(B))
    print_the_output(output_data)
    into_json(format_data(output_data))


def abnormality_check(B) -> dict:
    B_red = B.LLL()
    v_min = shortestVector(B_red)[1]
    x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
    G = gram_matrix(B)
    for i in range(G.nrows()):
        data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) # terrible
        if data != None:
            return data
    
main()

print("Done")

[  7  -5   8]
[  8  -6   8]
[-10   7 -10] :  B
[ 138  150 -185]
[ 150  164 -202]
[-185 -202  249] :  G
(1, 4, 4) :  The linear combination given by LLL
(1.0000, 0.62500, 1.2500) :  The linear combination suggested by cube
(0.00000, 3.3750, 2.7500) :  The difference
(-1, -1, 0) :  Shortest vector given by LLL
(-0.50000, 0.00000, 0.50000) :  Shortest vector given by cube

[ 0  6 -2]
[-1 -9  4]
[ 2 -5  2] :  B
[ 40 -62 -34]
[-62  98  51]
[-34  51  33] :  G
(4, 2, 1) :  The linear combination given by LLL
(2.2368, 0.89474, 1.0000) :  The linear combination suggested by cube
(1.7632, 1.1053, 0.00000) :  The difference
(0, 1, 2) :  Shortest vector given by LLL
(1.1053, 0.36841, 1.1053) :  Shortest vector given by cube

[ 8  6 -5]
[ 9  9 -7]
[-6 10 -8] :  B
[125 161  52]
[161 211  92]
[ 52  92 200] :  G
(10, -8, 1) :  The linear combination given by LLL
(8.4581, -6.8899, 1.0000) :  The linear combination suggested by cube
(1.5419, -1.1101, 0.00000) :  The difference
(2, -2, -2) :  Shortest ve

In [41]:
mat = matrix([[ -1 ,  9,   1],
           [ -1 ,  6 , -4],
           [  2, -10,   8]])

dictionary = abnormality_check(mat)
for pair in dictionary.items():
    print(pair[1], ": ", pair[0])


[ -1   9   1]
[ -1   6  -4]
[  2 -10   8] :  B
[ 83  51 -84]
[ 51  53 -94]
[-84 -94 168] :  G
(1, -8, -4) :  The linear combination given by LLL
(1.0000, -9.8824, -5.0294) :  The linear combination suggested by cube
(0.00000, 1.8824, 1.0294) :  The difference
(-1, 1, 1) :  Shortest vector given by LLL
(-1.1765, 0.00000, 0.29413) :  Shortest vector given by cube
1.2367 :  diff*B.norm
1.7320 :  LLL.norm
1.2127 :  cube.norm


### batshit cool math

In [2]:
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) -> dict:
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if difference > sensitivity: 
            if all([sol == 0 for sol in solutions]):
                break  
            return into_dict(B, G, x, solutions)
        
def find_real_minimum(G, i, x) -> vector:
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 



### output formatting

In [3]:
import json

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else:
            # FIXME here I assume that output_data consits only of matrices and one-line entries (vectors, free modules)
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open(jsonfilename, "w") 
    json.dump(output_data, out_file) 
    out_file.close() 


def print_the_output(output_data) -> None:
    for dict in output_data:
        for pair in dict.items():
            print(pair[1], ": ", pair[0])
        print()

### uninteresting tools

In [53]:
from random import randint, seed


def shortestVector(matrix): 
    """
    Returns the norm of the shortest vector and its index.
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    """
    Calculate Gram matrix of given matrix.
    :param matrix: given matrix
    :return: Gram matrix of matrix
    """
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]


def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["The linear combination given by LLL"] = x
    result["The linear combination suggested by cube"] = solutions
    result["The difference"] = x - vector(solutions)
    result["Shortest vector given by LLL"] = x*B
    result["Shortest vector given by cube"] = vector(solutions)*B
#     result["LLL.norm"] = vector(x*B).norm().n(digits=5)
#     result["cube.norm"] = (vector(solutions)*B).norm().n(digits=5)
    return result